In [6]:
#from IPython.display import Image
#Image(filename='i_could_care_less.png') 
#<div style="text-align:center"> <img src="i_could_care_less.png" width="700" height="700"> </div>
from IPython.display import Image
from IPython.display import HTML
html1 = '<div style="text-align:center"> <img src="i_could_care_less.png" width="700" height="700"> </div>'
HTML(html1)

#from IPython.display import Image
#from IPython.core.display import HTML 
#Image(filename= "i_could_care_less.png",width=500, height=500)

# Word Representation With One-Hot Vectors

One-hot encoding is the most common, most basic way to turn a token into a vector. It consists in associating a unique integer index to every word, then turning this integer index $i$ into a binary vector of size $V$, which is the size of our vocabulary, that would be all-zeros except for the $i$-th entry, which would be 1.

In [28]:
import numpy as np
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1

sentence = "the cat ate the god"
seq = []
for word in sentence.split():
    one_hot = np.zeros((1,len(token_index)))
    if word not in token_index:
        seq.append(one_hot.squeeze())
    else:
        one_hot[0,token_index[word]-1] = one_hot[0,token_index[word]-1] + 1
        seq.append(one_hot.squeeze())

In [29]:
seq

[array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [ ]:
import numpy as np
with open('cats100.test.txt') as file_handler:
    data = file_handler.read()

token_idx = {}
for word in data.split():
    if word not in token_idx:
        token_idx[word] = len(token_idx) + 1

sentence = "Geçtiğimiz zamanlarda kötü olaylar yaşandı"
one_hot_sentence = np.zeros((len(sentence.split()),len(token_idx)),dtype=np.int32)
for word_idx,word in enumerate(sentence.split()):
    one_hot_sentence[word_idx,token_idx[word] - 1] = 1

<figure>
  <img src="one-hot.png" alt="Trulli" width="300" height="300">
  <figcaption>Figure is taken from <a href="https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks">Stanford cs-230 cheatsheet</a> </figcaption>
</figure>

 There are two major issues with this approach. First issue is the curse of dimensionality, which refers to all sorts of problems that arise with data in high dimensions. This requires exponentially large memory space. Most of the matrix is taken up by zeros, so useful data becomes sparse. Imagine we have a vocabulary of 50,000. (There are roughly a million words in English language.) Each word is represented with 49,999 zeros and a single one, and we need 50,000 squared = 2.5 billion units of memory space. Not computationally efficient.
   

Second issue is that every one hotted vector is orthogonal to each other. You cannot measure the
similarity (like cosine similarity) on this vectors.


Those vectors can be visualized in 3 or 2-dimensional space as an example. In $\mathbb{R}^3$ (in other words, vocabulary size of 3, $\mid V \mid=3$), the word vectors become our span set which is 
$\left\{\begin{bmatrix} 1\\0\\0\end{bmatrix},
\begin{bmatrix} 0\\1\\0\end{bmatrix},
\begin{bmatrix} 0\\0\\1\end{bmatrix}\right\}$. The span set is linearly independent set. Therefore we cannot compute similarity metrics. The vectors $\vec{u_1}, \vec{u_2}, \vec{u_3}$ are orthogonal. Similarity metrics gives the result of zero. For example the cosine similarity:
<br>
    <br>
    

$$ \text{cos-sim}(\vec{u_1},\vec{u_2}) = \frac{\langle \vec{u_1}, \vec{u_2} \rangle}{\lVert \vec{u_1} \lVert_2 \times \lVert \vec{u_2} \lVert_2 } = \frac{\sum_{i=0}^n u_{1_i} \times u_{2_i}}{\sqrt{\sum_{i=0}^n u_{1_i}^2} \times \sqrt{\sum_{i=0}^n u_{2_i}^2}} = \frac{1 \times 0 + 0 \times 1 + 0 \times 0}{1 + 1} = 0$$
<br>
    


So, how to deal with those problems?

# Lexical Semantics And Distributional Linguistics


Words that occur in <em>similar contexts</em> tend to have similar meanings. The link between similarity and words are distributed and similarity in what they mean is called <strong>distributional hypothesis</strong> or <strong>distributional semantics</strong> in the field of Computational Linguistics. So what can be counted when we say similar contexts? For example if you surf on the Wikipedia page of linguistics, the words in this page somehow related with each other in the context of linguistics. This was formulated firsty by Martin Joos (Description of Language Design, 1950), Zellig Harris (Distributional Structure, 1954), John Rupert Firth (Applications of General Linguistics, 1957).


Some words have similar meanings, for example word cat and dog <strong>similar</strong>. Also, words can be <strong>antonyms</strong>, for example hot and cold. And words have <strong>connotations</strong> (TR: çağrışım), for example happy->positive connotation and sad->negative connotation. Can you feel the similarity of words, [study, exam, night, FF]?


Also each word can have multiple meanings. The word mouse can refeer to the rodent or the cursor control device. We call each of these aspects of the meaning of mouse a <strong>word sense</strong>. In other words words can be <strong>polysemous</strong> (have multiple senses), which can lead us to make word interpretations difficult! 
<br> <br>

- <strong>Word Sense Disambiguation</strong>: "Mouse info" (person who types this into a web search engine, looking for a pet info or a tool?) (determining which sense of a word is being used in a particular context)

The word **similarity** is very useful in larger semantics tasks. Knowing how similar two words are can help in computing how similar the meaning of two phrases or sentences are, a very important component of natural language understanding tasks like **question answering**, **summarization** etc.

| Word1       | Word2       |  Similarity (0-10) |
| ----------- | ----------- |  -----------       |
| Vanish      | Disappear   |  9.8             |
| Behave      | Obey        |  7.3              |
| Belief      | Impression  |  5.95              |
| Muscle      | Bone        |  3.65              |
| Modest      | Flexible    |  0.98              |
| Hole        | Agreement   |  0.3              |

We should look at **word relatedness** , the meaning of two words can be related in ways other than similarity. One such class of connections is called word **relatedness**, also traditionally called word **association** in pysch.

- The word *cup* and *coffee*.
- The word *inzva* and *deep learning*.

Also words can affective meanings. Osgood et al. 1957, proposed that words varied along three important dimensions of affective meaning: *valence, arousal, dominance*.
- **valence**: the pleasantness of the stimulus.
- **arousal**: the intensity of emotion provoked by the stimulus.
- **dominance**: the degree of control exerted by the stimulus.

Examples: 
- happy(1) $\uparrow$, satisfied(1) $\uparrow$; annoyed(1) $\downarrow$, unhappy(1)$\downarrow$
- excited(2) $\uparrow$, frenzied(2) $\uparrow$; relaxed(1) $\downarrow$, calm(2) $\downarrow$
- important(3)$\uparrow$, controlling(3)$\uparrow$; awed(3)$\downarrow$, influenced(3)$\downarrow$

*Question: does word embeddings has these dimesions?*

# Word Embeddings

How can we build a computational model that successfully deals with the different aspects of word meaning we saw above (word senses, word similarity, word relatedness, connotation etc.)?

**Instead of representing words with one-hot vector, sparse; with word embeddings we represents words with dense vectors**. 

**The idea of vector semantics is thus to represent a word as a point in some multidimensional semantic space.** Vectors for representing words are generally called **embeddings**. We **LEARN**  this embeddings form an arbitrary context.

<div style="text-align:center">
<figure>
  <img src="embed-dims.png" alt="Trulli" width="400" height="400">
  <figcaption>Figure is taken from <a href="https://medium.com/@hari4om/word-embedding-d816f643140">this medium post</a> </figcaption>
</figure>
</div>

Main two advantages of this word embeddings are:

- Now we represent words with dense vectors, which leads low memory requirements.

<figure>
  <img src="embed2.png" alt="Trulli" width="300" height="300">
  <figcaption>Figure is taken from <a href="https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks">Stanford cs-230 cheatsheet</a> </figcaption>
</figure>

- We can now calculate similarity metrics on these vectors!

## Visualizing Word Embeddings

Word embeddings can be visualized with various dimensionality reduction/matrix factorization algorithms.

<p float="left">

  <img src="socher.png" width="600" />
      
  <img src="tsne2.png" width="1000" /> 
</p>


- left-figure source: [Zero-Shot Learning Through Cross-Modal Transfer (Socher et al., 2013, NeurIPS)](https://nlp.stanford.edu/~socherr/SocherGanjooManningNg_NIPS2013.pdf)
- right-figure source: [Word representations: A simple and general method for semi-supervised learning (Turian et al., 2010)](http://metaoptimize.s3.amazonaws.com/cw-embeddings-ACL2010/embeddings-mostcommon.EMBEDDING_SIZE=50.png)

<p float="left">
  <img style="padding: 90px" src="colobert.png" width="800" />
      
  <img style="padding: 80px" src="bilingual.png" width="500" /> 
</p>


- left-figure source: [Natural Language Processing (almost) from Scratch (Collobert et al., 2011)](https://arxiv.org/abs/1103.0398v1.pdf)
- right-figure source: [Bilingual Word Embeddings for Phrase-Based Machine Translation (Socher et al., 2013, EMNLP)](https://ai.stanford.edu/~wzou/emnlp2013_ZouSocherCerManning.pdf)

## Using Word Embeddings

### Named Entity Recognition (NER)
In Natural language processing, Named Entity Recognition (NER) is a process where a sentence or a chunk of text is parsed through to find entities that can be put under categories like names, organizations, locations, quantities, monetary values, percentages, etc. Traditional NER algorithms included only names, places, and organizations.

Since the embeddings can capture word senses etc., it is practical and beneficial to use word embeddings in NER task. Embeddings can capture entity informations while capturing word relations.

<p float="left">
    <img src="ner2.png" width="700" /> 
  <img style="padding: 40px" src="ner1.png" width="600" />
</p>


figure sources: [link](https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede)

### Transfer Learning

- To learn word embeddings, huge size of training data is always useful. For example GloVe is trained on 5 separate corpora:
    * 2010 Wikipedia dump with 1 billion tokens
    * 2014 Wikipedia dump with 1.6 billion tokens
    * Gigaword 5 which has 4.3 billion tokens
    * the combination Gigaword5 + Wikipedia2014, which has 6 billion tokens
    * 42 billion tokens of web data, from Common Crawl
<br><br>

- You can download pre-trained word embeddings online.
    * [GloVe pre-trained vectors](https://nlp.stanford.edu/projects/glove/)
        * Common Crawl (42B tokens, 1.9M vocab, uncased, 300d vectors, 1.75 GB download)
        * Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download)
        * Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, 1.42 GB download)
    * [Hellinger PCA vectors](http://lebret.ch/words/)
    * [word2vec pre-trained vectors](https://wikipedia2vec.github.io/wikipedia2vec/pretrained/)
    * etc.
    <br><br>
- Transfer embedding to new task with smaller training set.
    * Language Modelling
    * Predictive Typing
    * Spelling/Grammar Correction
    * Summarization
    * NMT
    * etc.
<br><br>

- Finetune the word embeddings with new data (if your training data relatively big).


### Dependency Parsing
A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between "head" words and words which modify those heads. The figure below shows a dependency parse of a short sentence.

Syntactic Parsing or Dependency Parsing is the task of recognizing a sentence and assigning a syntactic structure to it. The most widely used syntactic structure is the parse tree which can be generated using some parsing algorithms. These parse trees are useful in various applications like grammar checking or more importantly it plays a critical role in the semantic analysis stage.

Dependency parsing is the task of analyzing the syntactic dependency structure of a given input sentence $S$. The output of a dependency parser is a dependency tree where the words of the input sentence are connected by typed dependency relations. Formally, the dependency parsing problem asks to create a mapping from the input sentence with words $S = w_0w_1...w_n$ (where $w_0$ is the ROOT) to its dependency tree graph $G$.

<p float="left">
   <img style="padding: 80px" src="dp.png" width="500" /> 
  <img style="padding: 40px" src="dp2.png" width="600" />
</p>

[A Fast and Accurate Dependency Parser using Neural Networks (Chen et al., 2014)](https://www.aclweb.org/anthology/D14-1082/)

right-figure source [CS224n Part IV](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes04-dependencyparsing.pdf)

### Representation Learning

Various tasks can be modeled with representations. Since words can be represented by embeddings, images can be (or even signals/audio) represented by a latent space $z$. Autoencoders is a way to learn latent representation of data.
<div style="text-align:center">

</div>
<p float="left">
    <img style="padding: 40px" src="ae3.png" width="500" /> 
   <img style="padding: 40px" src="ae1.png" width="450" /> 
  <img style="padding: 40px" src="ae2.png" width="450" />
</p>

## Semantic Properties Of Embeddings

Word embeddings have very important property: analogies. Analogy is another semantic property of embeddings that can capture relational meanings. Simply in words, analogy is to find **X**:

- **A is to B as C is to X**

For example **“woman is to queen as man is to X**. In this example **X** should be the word *king*.

Interestingly, such embeddings exhibit seemingly linear behaviour in analogies. This linear behaviour can be formulated as

- $w_a$ is to $w_a'$ as $w_b$ is to $w_b'$ $\rightarrow \rightarrow \rightarrow$ $w_a' - w_a + w_b \approx w_b'$

- vec('*queen*') - vec('*woman*') + vec('*man*') $\approx$ vec('*king*')

or

- vec('*queen*') - vec('*woman*') $\approx$ vec('*king*') - vec('*man*')


Another example:

- vec('*Paris*') - vec('*France*') $\approx$ vec('*Italy*') - vec('*Rome*')



<p float="left">
   <img style="padding: 80px" src="mikolov1.png" width="200" /> 
  <img style="padding: 30px" src="analogy1.png" width="500" />
     <img style="padding: 50px" src="mikolov2.png" width="600" />
</p>

- left-figure source: [Linguistic Regularities in Continuous Space Word Representations (Mikolov et al., 2013)](https://www.aclweb.org/anthology/N13-1090.pdf)
- mid-figure source: [Speech and Language Processing, Daniel Jurafsky, Third Edition](https://web.stanford.edu/~jurafsky/slp3/)
- right-figure source: [Efficient Estimation of Word Representations in Vector Space (Mikolov etl al., 2013)](https://arxiv.org/pdf/1301.3781.pdf)

Do vector embeddings capture syntactic relationships? Yes. Capture with linear behaviours? Yes.


<p float="left">
   <img style="padding: 80px" src="syntax2.png" width="200" /> 
  <img style="padding: 30px" src="syntax1.png" width="500" />
</p>

- left-figure source: [Linguistic Regularities in Continuous Space Word Representations (Mikolov et al., 2013)](https://www.aclweb.org/anthology/N13-1090.pdf)
- right-figure source: [Speech and Language Processing, Daniel Jurafsky, Third Edition](https://web.stanford.edu/~jurafsky/slp3/)


For more formal definition of analogy? Check [Analogies Explained: Towards Understanding Word Embeddings (Allen et al., 2019)](https://arxiv.org/pdf/1901.09813.pdf)

## Evaluating Word Vectors

- Extrinsic Evaluation
    * This is the evaluation on a real task.
    * Can be slow to compute performance.
    * Unclear if the subsystem is the problem, or our system.
    * If replacing subsystem improves performance, the change is likely good.
    * NER, Question Answering etc.


- Intrinsic Evaluation
    * Fast to compute
    * Helps to understand subsystem
    * Needs positive correlation with real task to determine usefulness
    
    
SimLex-999 dataset (Hill et al., 2015) gives values on a scale from 0 to 10 by asking humans to judge how similar one word is to another. Other datasets for evaluating word vectors: 
- WordSim 353 
- TOEFL Dataset
- SCWS Dataset
- Word-in-Context (WiC)
- Miller & Charles Dataset
- Rubenstein & Goodenough Dataset
- Stanford Rare Word (RW)

<p float="left">
   <img src="glove-table.png" width="400" /> 
  <img style="padding: 100px" src="glove-table2.png" width="400" />
</p>

Some evaluation metrics from [GloVe: Global Vectors for Word Representation (Pennington et al., 2014)](https://www.aclweb.org/anthology/D14-1162.pdf)

# Learning Word Embeddings

## Embedding Matrix
Embedding matrix can be represented as a single matrix $E \in \mathbb{R}^{d \times \mid V \mid}$ (or $E \in \mathbb{R}^{ \mid V \mid \times d }$, it does not even matter), where d is embedding size and $\mid V \mid$ is size of the vocabulary $V$.

<br>

$$E = \begin{bmatrix}
\text{hello} & \text{i} & \text{love} & \text{inzva} & \cdots & \text{sanctuary}\\
0.1 & 0.137 & -0.03 & -0.44 & \cdots & 0.36\\
-0.78 & -0.25 & 2.09 & 0.19 & \cdots & 0.32\\
-3.1 & 1.54 & -2.52 & 0.2 & \cdots & -1.51\\
1.13 & -0.78 & -0.56 & 0.95 & \cdots & 0.2112\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
- 0.6 & 0.13 & 3.89 & -0.071 & -0.27 & 0.27 
\end{bmatrix} \in \mathbb{R}^{d \times \mid V \mid}$$

But how to learn them?

## A Neural Probabilistic Language Model (Bengio et al., 2003)

*In neural language models, the prior context is represented by embeddings of the previous words*. Representing the prior context as embeddings, rather than by exact words as used in [n-gram](https://web.stanford.edu/~jurafsky/slp3/3.pdf) language models, allows neural language models to generalize to unseen data much better than n-gram language models.

For example in our training set we see this sentence,

    Today, after school, I am planning to go to cinema.
    
but we have never seen the word "concert" after the words "go to". In our test set we are trying to predict what comes after the prefix "Today, after school, I am planning to go to". An n-gram language model will predict "cinema" but not "concert". But a neural language model, which can make use of the fact that "cinema" and "concert" have similar embeddings, will be able to assign a reasonably high probability to "concert" as well as "cinema", merely because they have similar vectors.

In 2003 Bengio proposed a neural language model that can learns and uses embeddings to predict the next word in a sentence. Formally representation, for a sequence of words

$$x^{(1)}, x^{(2)}, ..., x^{(t)}$$

the probability distribution of the next word (output) is

$$p(x^{(t+1)} \mid x^{(1)}, x^{(2)}, ..., x^{(t)})$$

Bengio proposed a fixed-window neural Language Model which can be seen as the same approach of n-grams.

<div style="text-align:center">
"<s> as the proctor started the clock</s> the students opened their ____."
</div>
<br>
We have a moving window at time $t$ with an embedding vector representing each of the window size previous words. For window size 3, words $w_{t-1}, w_{t-2}, w_{t-3}$. These 3 vectors are concatenated together to produce input x. The task is to predict $w_t$.

<p float="left">
   <img src="neural.png" width="400" /> 
  <img style="padding: 40px" src="neural2.png" width="800" />
</p>

- left-figure source: [CS224n lecture 5](http://web.stanford.edu/class/cs224n/slides/cs224n-2021-lecture05-rnnlm.pdf)
- right-figure source: [Speech and Language Processing, Daniel Jurafsky, Third Edition](https://web.stanford.edu/~jurafsky/slp3/)

For this task, we are going to represent each of the $N$ prevuios words as a one-hot-vector of length $\mid V \mid$.

The forward equation for neural language model

- Input $x_i \in R^{1 \times \mid V \mid}$

- Learning word embeddings: $e = concat(x_1 E^T, x_2 E^T, x_3 E^T)$
    * $E \in \mathbb{R}^{d \times \mid V \mid}$
    * $e \in \mathbb{R}^{1 \times 3d}$
    
    
- $h = \sigma(e W^T + b_1)$
    * $W \in \mathbb{R}^{d_h \times 3d}$
    * $h \in \mathbb{R}^{1 \times d_h}$
    
    
- $z = h U^T + b_2$
    * $U \in \mathbb{R}^{\mid V \mid \times d_h}$
    * $z \in \mathbb{R}^{1 \times \mid V \mid}$
    
    
- $\hat{y} = softmax(z) \in \mathbb{R}^{1 \times \mid V \mid}$


Then the model is trained, at each word $w_t$, the negative log likelihood loss is:

$$ L = - \log p(w_t \mid w_{(t-1)}, w_{(t-2)}, ..., w_{(t-n+1)}) = softmax(z) $$

<br>
$$ \theta_{t+1} = \theta_t - \eta \frac{\partial - \log p(w_t \mid w_{(t-1)}, w_{(t-2)}, ..., w_{(t-n+1)})}{\partial \theta}$$

# word2vec

word2vec is proposed in the paper called [Distributed Representations of Words and Phrases and their Compositionality (Mikolov et al., 2013)](https://arxiv.org/abs/1310.4546). It allows you to learn the high-quality distributed vector representations that capture a large number of precise syntactic and semantic word relationships. Word2vec algorithm uses Skip-gram [Efficient Estimation of Word Representations in Vector Space
 (Mikolov et al., 2013)](https://arxiv.org/abs/1301.3781) model to learn efficient vector representations. Those learned word vectors has interesting property, words with semantic and syntactic affinities give the necessary result in mathematical similarity operations.
 
Suppose that you have a sliding window of a fixed size moving along a sentence: the word in the middle is the “target” and those on its left and right within the sliding window are the context words.

The skip-gram model is trained to predict the probabilities of a word being a context word for the given target.

<p float="left">
   <img style="padding: 40px" src="sent1.png" width="600" /> 
  <img style="padding: 40px" src="sent2.png" width="600" />
</p>


For example consider this sentence,

         "A change in Quantity also entails a change in Quality"
Our target and context pairs for window size of 5:


| Sliding window (size = 5)       | Target word       |  Context |
| -----------                     | -----------       |  -----------       |
| \[A change in\]                 | a                 |  change, in            |
| \[A change in Quantity \]       | change        |  a, in, quantitiy             |
| \[A change in Quantity  also\]       | in  | a, change, quantitiy,also               |
| ...      |  ...      |  ...          |
| \[entails a change in Quality\]      | change    |  entails, a, in, Quality             |
| \[a change in Quality\]              | in   |  a, change, Quality             |
| \[change in Quality\]              | quality   |  change, in             |

Each context-target pair is treated as a new observation in the data. 

For each position $t=1,..,T$ predict context words within a window of fixed size $m$, given center word $w_j$. In Skip-gram connections we have an objective to maximize, likelihood (or minimize log-likelihood):

<br>
$$\max \limits_{\theta} \prod_{\text{center}} \prod_{\text{context}} p(\text{context}|\text{center} ;\theta)$$

<br>
$$= \max \limits_{\theta} \prod_{t=1}^T \prod_{-c \leq j \leq c, j \neq c} p(w_{t+j}|w_t; \theta)$$

<br>
$$= \min \limits_{\theta} -\frac{1}{T} \prod_{t=1}^T \prod_{-c \leq j \leq c, j \neq c} p(w_{t+j}|w_t; \theta)$$

<br>
$$= \min \limits_{\theta} -\frac{1}{T} \sum_{t=1}^T \sum_{-c \leq j \leq c, j \neq c} \log p(w_{t+j}|w_t; \theta)$$

So, How can we calculate those probabilities? Softmax gives the normalized probabilities.

## Parameterization Of Skip-Gram Model

Let's say $w_t$ is our target word and $w_c$ is current context word. The softmax is defined as

<br>
$$p(w_c \mid w_t) = \frac{\exp(v_{w_c}^T v_{w_t})}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})}$$

maximizing this log-likelihood function under $v_{w_t}$ gives you the most likely value of the $v_{w_t}$ given the data.

<br>
$$ \frac{\partial}{\partial v_{w_t}}\cdot \log \frac{\exp(v_{w_c}^T v_{w_t})}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})}$$

<br>

$$ = \frac{\partial}{\partial v_{w_t}}\cdot \log \underbrace{\exp(v_{w_c}^T v_{w_t})}_{\text{numerator}} - \frac{\partial}{\partial v_{w_t}}\cdot \log \underbrace{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})}_{\text{denominator}}$$

<br>

$$ \frac{\partial}{\partial v_{w_t}} \cdot v_{w_c}^T v_{w_t} =  v_{w_c} \; \; (\text{numerator})$$

Now, it is time to derive denominator.
<br>

$$\frac{\partial}{\partial v_{w_t}}\cdot \log \sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t}) = \frac{1}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})} \cdot \frac{\partial}{\partial v_{w_t}} \sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})$$

<br>

$$ = \frac{1}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})} \cdot  \sum_{i=0}^{\mid V \mid} \frac{\partial}{\partial v_{w_t}} \cdot \exp(v_{w_i}^T v_{w_t})$$

<br>

$$ = \frac{1}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})} \cdot  \sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t}) \frac{\partial}{\partial v_{w_t}} v_{w_i}^T v_{w_t}$$

<br>

$$ = \frac{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t}) \cdot v_{w_i}}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})} \;\; (\text{denominator}) $$


To sum up,

$$\frac{\partial}{\partial w_t} \log p(w_c \mid w_t) = v_{w_c} - \frac{\sum_{j=0}^{\mid V \mid} \exp(v_{w_j}^T v_{w_t}) \cdot v_{w_j}}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})}$$

<br>

$$ = v_{w_c} - \sum_{j=0}^{\mid V \mid} \frac{\exp(v_{w_j}^T v_{w_t})}{\sum_{i=0}^{\mid V \mid} \exp(v_{w_i}^T v_{w_t})} \cdot v_{w_j}$$

<br>

$$ \underbrace{= v_{w_c} - \sum_{j=0}^{\mid V \mid} p(w_j \mid w_t) \cdot v_{w_j}}_{\nabla_{w_t}\log p(w_c \mid w_t)}$$

This is the observed representation subtract $\mathop{\mathbb{E}}[w_j \mid w_t]$.

## Negative Sampling (Noise Contrastive Estimation (NCE))

The Noise Contrastive Estimation (NCE) metric intends to differentiate the target word from noise samples using a logistic regression classifier [(Noise-contrastive estimation: A new estimation principle for unnormalized statistical models, Gutmann et al., 2010)](http://proceedings.mlr.press/v9/gutmann10a/gutmann10a.pdf). 

In softmax computation, look at the denominator. The summation over $\mid V\mid$ is computationally expensive. The training or evaluation takes asymptotically $O(\mid V \mid)$. In a very large corpora, the most frequent words can easily occur hundreds or millions of times ("in", "and", "the", "a" etc.). Such words provides less information value than the rare words. For example, while the skip-gram model benefits from observing co-occurences of "inzva" and "deep learning", it benefits much less from observing the frequent co-occurences of "inzva" and "the".In a very large corpora, the most frequent words can easily occur hundreds or millions of times ("in", "and", "the", "a" etc.). Such words provides less information value than the rare words. For example, while the skip-gram model benefits from observing co-occurences of "inzva" and "deep learning", it benefits much less from observing the frequent co-occurences of "inzva" and "the". 

For every training step, instead of looping over the entire vocabulary, we can just sample several negative examples! We "sample" from
a noise distribution $P_n(w)$ whose probabilities match the ordering of the frequency of the vocabulary.

Consider a pair $(w_t, w_c)$ of word and context. Did this pair come from the training data? Let’s denote by $p(D=1 \mid w_t,w_c)$ the probability that $(w_t, w_c)$ came from the corpus data. Correspondingly $p(D=0 \mid w_t,w_c)$ will be the probability that $(w_t, w_c)$ didn't come from the corpus data. First, let’s model $p(D=1 \mid w_t,w_c)$ with sigmoid:
<br>

$$p(D=1 \mid w_t,w_c) = \sigma(v_{w_c}^T v_{w_t}) = \frac{1}{1 + \exp(- v_{w_c}^T v_{w_t})}$$

Now, we build a new objective function that tries to maximize the probability of a word and context being in the corpus data if it indeed is, and maximize the probability of a word and context not being in the corpus data if it indeed is not. Maximum likelihood says:

$$ \max \prod_{(w_t, w_c) \in D} p(D=1 \mid w_t,w_c) \times \prod_{(w_t, w_c) \in D'} p(D=0 \mid w_t,w_c)$$

<br>

$$ = \max \prod_{(w_t, w_c) \in D} p(D=1 \mid w_t,w_c) \times \prod_{(w_t, w_c) \in D'} 1 - p(D=1 \mid w_t,w_c)$$

<br>

$$ = \max \sum_{(w_t, w_c) \in D} \log p(D=1 \mid w_t,w_c) + \sum_{(w_t, w_c) \in D'}  \log (1 - p(D=1 \mid w_t,w_c))$$

<br>

$$ = \max \sum_{(w_t, w_c) \in D} \log \frac{1}{1 + \exp(- v_{w_c}^T v_{w_t})} + \sum_{(w_t, w_c) \in D'} \log \left(1 - \frac{1}{1 + \exp(- v_{w_c}^T v_{w_t})}\right)$$

<br>

Note that $\frac{\exp(-x)}{(1 + \exp(-x))} \times \frac{\exp(x)}{\exp(x)} = \frac{1}{(1 + \exp(x))}$


$$ = \max \sum_{(w_t, w_c) \in D} \log \frac{1}{1 + \exp(- v_{w_c}^T v_{w_t})} + \sum_{(w_t, w_c) \in D'} \log \frac{1}{1 + \exp(v_{w_c}^T v_{w_t})}$$

Maximizing the likelihood is the same as minimizing the negative log likelihood:

<br>

$$L = - \sum_{(w_t, w_c) \in D} \log \frac{1}{1 + \exp(- v_{w_c}^T v_{w_t})} -  \sum_{(w_t, w_c) \in D'} \log \frac{1}{1 + \exp(v_{w_c}^T v_{w_t})}$$

Note that $D'$ is a "false" or "negative" corpus. Where we would have sentences like "the school is eaten by pilgrims". Unnatural sentences that should get a low probability of ever occurring. We can generate $D'$ on the fly by randomly sampling this negative from the word bank.

The Negative Sampling (NEG) proposed in the original word2vec paper. NEG approximates the binary classifier’s output with sigmoid functions as follows:

$$\begin{align}
p(d=1 \vert v_{w_c}, v_{w_t}) &= \sigma(v_{w_c}^T v_{w_t}) \\
p(d=0 \vert v_{w_c}, v_{w_t}) &= 1 - \sigma(v_{w_c}^T v_{w_t}) = \sigma(-v_{w_c}^T v_{w_t})
\end{align}$$

So the objective is

$$L = - [ \log \sigma(v_{w_c}^T v_{w_t}) +  \sum_{\substack{i=1 \\ \tilde{w}_i \sim Q}}^K \log \sigma(v_{\tilde{w}_i}^T v_{w_t})]$$

In the above formulation, ${v_{\tilde{w}_i} \mid i = 1 . . . K}$ are sampled from $P_n(w)$. How to define $P_n(w)$? In the word2vec paper $P_n(w)$ defined as 

$$P_n(w_i) = 1 - \sqrt{\frac{t}{freq(w_i)}} \;\; t \approx 10^{-5}$$

This distribution assigns lower probability for lower frequency words, higher probability for higher frequency words.

Hence, this distribution is sampled form a unigram distribution $U(w)$ raised to the  $\frac{3}{4}$rd power.

The unigram distribuiton is defined as

$$P_n(w)= \left(\frac{U(w)}{Z}\right)^\alpha$$

Or just by Andrew NG's definition:

$$P_n(w_i) = \frac{freq(w_i)^\frac{3}{4}}{\sum_{j=0}^M freq(w_j)^\frac{3}{4}}$$

Raising the unigram distribution $U(w)$ to the power of $\alpha$ has an effect of smoothing out the distribution. It attempts to combat the imbalance between common words and rare words by decreasing the probability of drawing common words, and increasing the probability drawing rare words.

<div style="text-align:center"> <img src="noise_dist.png" width="800" height="800"> </div>

In [1]:
import numpy as np
unig_dist  = {'inzva': 0.023, 'deep': 0.12, 'learning': 0.34, 'the': 0.517}
sum(unig_dist.values())

1.0

In [2]:
alpha      = 3 / 4
noise_dist = {key: val ** alpha for key, val in unig_dist.items()}
Z = sum(noise_dist.values())
noise_dist_normalized = {key: val / Z for key, val in noise_dist.items()}
noise_dist_normalized

{'inzva': 0.044813853132981724,
 'deep': 0.15470428538870049,
 'learning': 0.33785130228003507,
 'the': 0.4626305591982827}

In [3]:
sum(noise_dist_normalized.values())

1.0

In [4]:
K = 10
np.random.choice(list(noise_dist_normalized.keys()), size=K, p=list(noise_dist_normalized.values()))

array(['the', 'the', 'deep', 'learning', 'the', 'learning', 'the',
       'inzva', 'the', 'learning'], dtype='<U8')

More on negative sampling? [link](https://www.aclweb.org/anthology/D17-1037.pdf).

Deriving negative sampling loss function is trivial. Left to you.

## Hierarchical Softmax


hierarchical softmax is proposed to make the sum calculation faster with the help of a binary tree structure. The model uses a binary tree to represent all words in the
vocabulary. The $\mid V \mid$ words must be leaf units of the tree.